# K7 Yukawa Tensor Analysis v1.0

Analysis of Yukawa couplings extracted from the GIFT-calibrated K7 metric.

**Source data**: `artifacts/meta_hodge/yukawa_tensor.pt`

**Geometry**: v1.6 pre-computed samples with:
- det(g) = 65/32 = 2.03125 (exact GIFT target)
- kappa_T ~ 1/61 = 0.0164 (torsion magnitude)

**Cohomology**:
- b2(K7) = 21 harmonic 2-forms
- b3(K7) = 35 harmonic 3-forms (single version slice)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
OUT_DIR = Path("../../artifacts/meta_hodge")
yukawa_path = OUT_DIR / "yukawa_tensor.pt"

# Load Yukawa tensor
Y = torch.load(yukawa_path, map_location="cpu")
print(f"Yukawa tensor shape: {Y.shape}")
print(f"Expected: (21, 21, 35) for Y_ijk = int omega_i ^ omega_j ^ Omega_k")

## 1. Sanity Checks

In [ ]:
# Check symmetry Y_ijk = Y_jik
sym_diff = (Y - Y.transpose(0, 1)).abs().max()
print(f"Symmetry check: max|Y_ijk - Y_jik| = {sym_diff:.2e}")
print(f"Symmetric: {sym_diff < 1e-10}")

In [ ]:
# Basic statistics
vals = Y.flatten().detach().cpu().numpy()
nonzero_mask = np.abs(vals) > 1e-10
nonzero = vals[nonzero_mask]

print(f"Total entries:  {vals.size}")
print(f"Non-zero:       {nonzero.size} ({100*nonzero.size/vals.size:.1f}%)")
print(f"Zero entries:   {vals.size - nonzero.size} ({100*(vals.size - nonzero.size)/vals.size:.1f}%)")
print(f"")
print(f"Min value:      {vals.min():.6f}")
print(f"Max value:      {vals.max():.6f}")
print(f"Mean:           {vals.mean():.6f}")
print(f"Std:            {vals.std():.6f}")
print(f"")
print(f"Non-zero min:   {np.abs(nonzero).min():.6f}")
print(f"Non-zero max:   {np.abs(nonzero).max():.6f}")
print(f"Non-zero mean:  {np.abs(nonzero).mean():.6f}")

## 2. Distribution of Yukawa Couplings

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale histogram
ax1 = axes[0]
ax1.hist(vals, bins=100, alpha=0.7, edgecolor='black')
ax1.set_xlabel("Y_ijk")
ax1.set_ylabel("Count")
ax1.set_title("Distribution of Yukawa couplings (linear)")
ax1.axvline(0, color='r', linestyle='--', alpha=0.5)

# Log scale histogram (absolute values)
ax2 = axes[1]
log_vals = np.log10(np.abs(nonzero) + 1e-15)
ax2.hist(log_vals, bins=50, alpha=0.7, edgecolor='black', color='orange')
ax2.set_xlabel("log10 |Y_ijk|")
ax2.set_ylabel("Count")
ax2.set_title("Distribution of Yukawa couplings (log scale)")

plt.tight_layout()
plt.show()

# Hierarchy analysis
sorted_abs = np.sort(np.abs(nonzero))[::-1]
print(f"\nTop 10 largest |Y_ijk|:")
for i, v in enumerate(sorted_abs[:10]):
    print(f"  {i+1}. {v:.4f}")

print(f"\nHierarchy ratio (max/median): {sorted_abs[0]/np.median(sorted_abs):.1f}x")
print(f"Hierarchy ratio (max/min):    {sorted_abs[0]/sorted_abs[-1]:.1f}x")

## 3. Top Yukawa Couplings

In [ ]:
# Find top couplings with indices
flat = Y.flatten()
topk = torch.topk(flat.abs(), 20)

print("Top 20 Yukawa couplings Y_ijk:")
print("="*50)
for rank, (idx, val) in enumerate(zip(topk.indices, topk.values), 1):
    i = idx // (21 * 35)
    j = (idx // 35) % 21
    k = idx % 35
    sign = '+' if flat[idx] > 0 else '-'
    print(f"{rank:3d}. Y[{i:2d},{j:2d},{k:2d}] = {sign}{val:.4f}")

## 4. Mode-by-Mode Analysis (H3 modes)

In [ ]:
def analyze_mode(k):
    """Analyze the 21x21 Yukawa matrix for H3 mode k."""
    M = Y[:, :, k]
    eigvals, _ = torch.linalg.eigh(M)
    eigvals = eigvals.detach().cpu().numpy()
    frob_norm = torch.linalg.norm(M).item()
    return {
        'k': k,
        'min_eig': eigvals.min(),
        'max_eig': eigvals.max(),
        'trace': eigvals.sum(),
        'frob_norm': frob_norm,
        'spectral_gap': eigvals.max() - eigvals[-2] if len(eigvals) > 1 else 0,
    }

mode_data = [analyze_mode(k) for k in range(35)]

print("Mode analysis (every 5th mode):")
print("="*70)
print(f"{'k':>4} {'min_eig':>12} {'max_eig':>12} {'trace':>12} {'||Y||_F':>12}")
print("-"*70)
for d in mode_data[::5]:
    print(f"{d['k']:4d} {d['min_eig']:12.4f} {d['max_eig']:12.4f} {d['trace']:12.4f} {d['frob_norm']:12.4f}")

In [ ]:
# Plot mode strength
mode_strength = np.array([d['frob_norm'] for d in mode_data])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Frobenius norm by mode
ax1 = axes[0]
ax1.bar(range(35), mode_strength, alpha=0.7)
ax1.set_xlabel("k (H3 mode)")
ax1.set_ylabel("||Y^{(k)}||_F")
ax1.set_title("Yukawa coupling strength by H3 mode")

# Eigenvalue spread by mode
ax2 = axes[1]
max_eigs = np.array([d['max_eig'] for d in mode_data])
min_eigs = np.array([d['min_eig'] for d in mode_data])
ax2.fill_between(range(35), min_eigs, max_eigs, alpha=0.3)
ax2.plot(range(35), max_eigs, 'b-', label='max eigenvalue')
ax2.plot(range(35), min_eigs, 'r-', label='min eigenvalue')
ax2.axhline(0, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel("k (H3 mode)")
ax2.set_ylabel("Eigenvalue")
ax2.set_title("Eigenvalue range by H3 mode")
ax2.legend()

plt.tight_layout()
plt.show()

# Identify dominant modes
sorted_modes = np.argsort(mode_strength)[::-1]
print(f"\nTop 5 dominant H3 modes:")
for i, k in enumerate(sorted_modes[:5]):
    print(f"  {i+1}. k={k}: ||Y^(k)||_F = {mode_strength[k]:.4f}")

## 5. Rank Analysis (Mass Hierarchy)

In [ ]:
def numerical_rank(M, tol=1e-6):
    """Compute numerical rank via SVD."""
    s = torch.linalg.svdvals(M)
    return int((s > tol).sum().item()), s.detach().cpu().numpy()

print("Numerical rank of Y^(k) matrices (tolerance = 1e-6):")
print("="*60)

ranks = []
for k in range(35):
    M = Y[:, :, k]
    r, svd = numerical_rank(M)
    ranks.append(r)

# Plot rank distribution
plt.figure(figsize=(10, 4))
plt.bar(range(35), ranks, alpha=0.7)
plt.xlabel("k (H3 mode)")
plt.ylabel("Numerical rank")
plt.title("Numerical rank of Yukawa matrices by H3 mode")
plt.axhline(3, color='r', linestyle='--', label='rank=3 (3 generations)')
plt.legend()
plt.show()

print(f"\nRank distribution:")
from collections import Counter
rank_counts = Counter(ranks)
for r, count in sorted(rank_counts.items()):
    print(f"  rank {r}: {count} modes")

In [ ]:
# Detailed SVD for dominant mode
dominant_k = sorted_modes[0]
M_dom = Y[:, :, dominant_k]
_, svd_dom = numerical_rank(M_dom)

print(f"SVD of most dominant mode (k={dominant_k}):")
print("="*50)
for i, s in enumerate(svd_dom[:10]):
    print(f"  sigma_{i}: {s:.6f}")

# Plot singular value decay
plt.figure(figsize=(10, 4))
plt.semilogy(svd_dom, 'o-')
plt.xlabel("Singular value index")
plt.ylabel("Singular value (log scale)")
plt.title(f"Singular value decay for mode k={dominant_k}")
plt.axhline(1e-6, color='r', linestyle='--', label='tol=1e-6')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Check for rank-3 structure (potential 3-generation signal)
if len(svd_dom) >= 3:
    ratio_12 = svd_dom[0] / svd_dom[1] if svd_dom[1] > 1e-10 else float('inf')
    ratio_23 = svd_dom[1] / svd_dom[2] if svd_dom[2] > 1e-10 else float('inf')
    ratio_34 = svd_dom[2] / svd_dom[3] if len(svd_dom) > 3 and svd_dom[3] > 1e-10 else float('inf')
    print(f"\nHierarchy ratios:")
    print(f"  sigma_0/sigma_1 = {ratio_12:.2f}")
    print(f"  sigma_1/sigma_2 = {ratio_23:.2f}")
    print(f"  sigma_2/sigma_3 = {ratio_34:.2f}")

## 6. Block Structure Analysis (21 = 3 x 7)

In [ ]:
def show_block_structure(k, block_size=7):
    """Visualize block structure of Y^(k) matrix."""
    M = Y[:, :, k].detach().cpu().numpy()
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Full matrix
    ax1 = axes[0]
    im1 = ax1.imshow(M, cmap='RdBu_r', aspect='equal')
    ax1.set_title(f"Y^({k}) full matrix (21x21)")
    ax1.set_xlabel("j")
    ax1.set_ylabel("i")
    plt.colorbar(im1, ax=ax1)
    
    # Add block grid lines
    for i in range(1, 3):
        ax1.axhline(i*block_size - 0.5, color='k', linewidth=0.5)
        ax1.axvline(i*block_size - 0.5, color='k', linewidth=0.5)
    
    # Block norms
    ax2 = axes[1]
    block_norms = np.zeros((3, 3))
    for bi in range(3):
        for bj in range(3):
            block = M[bi*block_size:(bi+1)*block_size, bj*block_size:(bj+1)*block_size]
            block_norms[bi, bj] = np.linalg.norm(block)
    
    im2 = ax2.imshow(block_norms, cmap='viridis', aspect='equal')
    ax2.set_title(f"Block norms (3x3 from 21=3x7)")
    ax2.set_xticks(range(3))
    ax2.set_yticks(range(3))
    ax2.set_xticklabels(['Block 0', 'Block 1', 'Block 2'])
    ax2.set_yticklabels(['Block 0', 'Block 1', 'Block 2'])
    
    for i in range(3):
        for j in range(3):
            ax2.text(j, i, f"{block_norms[i,j]:.2f}", ha='center', va='center', color='white')
    
    plt.colorbar(im2, ax=ax2)
    plt.tight_layout()
    plt.show()
    
    return block_norms

# Analyze block structure for top 3 dominant modes
print("Block structure analysis (21 = 3 x 7):")
print("="*60)
for k in sorted_modes[:3]:
    print(f"\nMode k={k}:")
    block_norms = show_block_structure(k)

## 7. Aggregate Yukawa Matrix

In [ ]:
# Sum over H3 modes to get aggregate Yukawa matrix
Y_agg = Y.sum(dim=2)
print(f"Aggregate Yukawa matrix shape: {Y_agg.shape}")

# Analyze aggregate
eigvals_agg, eigvecs_agg = torch.linalg.eigh(Y_agg)
eigvals_agg = eigvals_agg.detach().cpu().numpy()

print(f"\nAggregate matrix eigenvalues:")
print(f"  min: {eigvals_agg.min():.4f}")
print(f"  max: {eigvals_agg.max():.4f}")

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax1 = axes[0]
im1 = ax1.imshow(Y_agg.detach().cpu().numpy(), cmap='RdBu_r', aspect='equal')
ax1.set_title("Aggregate Yukawa matrix (sum over H3)")
ax1.set_xlabel("j")
ax1.set_ylabel("i")
plt.colorbar(im1, ax=ax1)

ax2 = axes[1]
ax2.bar(range(21), eigvals_agg, alpha=0.7)
ax2.set_xlabel("Eigenvalue index")
ax2.set_ylabel("Eigenvalue")
ax2.set_title("Eigenvalue spectrum of aggregate matrix")
ax2.axhline(0, color='k', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Summary Statistics

In [ ]:
print("="*70)
print("YUKAWA TENSOR ANALYSIS SUMMARY")
print("="*70)

print(f"\n1. TENSOR STRUCTURE")
print(f"   Shape: {Y.shape} (21 H2-modes x 21 H2-modes x 35 H3-modes)")
print(f"   Total entries: {Y.numel()}")
print(f"   Non-zero (>1e-6): {(Y.abs() > 1e-6).sum().item()} ({100*(Y.abs() > 1e-6).sum().item()/Y.numel():.1f}%)")
print(f"   Symmetry Y_ijk = Y_jik: {sym_diff < 1e-10}")

print(f"\n2. AMPLITUDE DISTRIBUTION")
print(f"   Max |Y_ijk|: {Y.abs().max():.4f}")
print(f"   Mean |Y_ijk|: {Y.abs().mean():.6f}")
print(f"   Hierarchy (max/median): {sorted_abs[0]/np.median(sorted_abs):.1f}x")

print(f"\n3. MODE ANALYSIS")
print(f"   Most dominant H3 mode: k={sorted_modes[0]} (||Y||_F = {mode_strength[sorted_modes[0]]:.4f})")
print(f"   Mode strength range: [{mode_strength.min():.4f}, {mode_strength.max():.4f}]")

print(f"\n4. RANK STRUCTURE")
print(f"   Rank distribution: {dict(rank_counts)}")
print(f"   Modes with rank >= 3: {sum(1 for r in ranks if r >= 3)}")

print(f"\n5. GIFT INVARIANTS (from source metric)")
print(f"   det(g) = 65/32 = 2.03125 (exact)")
print(f"   kappa_T ~ 1/61 = 0.0164")
print(f"   b2(K7) = 21, b3(K7) = 35 (single version)")

print("\n" + "="*70)